# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [56]:
#Your code here
import pandas as pd #to work with dataframes
import numpy as np #for any and all calculations
import sqlite3 #for working with SQL files to make dataframes
from scipy import stats


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

df = pd.read_csv("homepage_actions.csv", index_col=0, low_memory=False)
df

,id,group,action
timestamp,,,
2016-09-24 17:42:27.839496,804196,experiment,view
2016-09-24 19:19:03.542569,434745,experiment,view
2016-09-24 19:36:00.944135,507599,experiment,view
2016-09-24 19:59:02.646620,671993,control,view
2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...
2017-01-18 09:11:41.984113,192060,experiment,view
2017-01-18 09:42:12.844575,755912,experiment,view
2017-01-18 10:01:09.026482,458115,experiment,view


In [57]:
#importing flatiron stats the long form way

def welch_t(a, b):
    
    """ Calculate Welch's t statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. This function returns the degrees of freedom """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):
    """Calculates the p-value for Welch's t-test given two samples.
    By default, the returned p-value is for a one-sided t-test. 
    Set the two-sided parameter to True if you wish to perform a two-sided t-test instead.
    """
    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p
    
def Remove_Outlier_Indices(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    trueList = ~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)))
    return trueList


In [83]:
df_AB_ex = df[(df['group'] == 'experiment') &
           ((df['action'] == 'view')  | (df['action'] == 'click'))].reset_index(drop = True)

df_AB_con = df[(df['group'] == 'control') &
           ((df['action'] == 'view')  | (df['action'] == 'click'))].reset_index(drop = True)
print("Experimental Total:", sum(df_AB_ex['id'].value_counts()) - df_AB_ex['action'].value_counts()['click'])
print("Experimental:", df_AB_ex['action'].value_counts())

print("Control Total:", sum(df_AB_con['id'].value_counts()) - df_AB_con['action'].value_counts()['click'])
print("Control:", df_AB_con['action'].value_counts())

Experimental Total: 2996
Experimental: view     2996
click     928
Name: action, dtype: int64
Control Total: 3332
Control: view     3332
click     932
Name: action, dtype: int64


In [84]:
# assigning names to counts
#EXPERIMENT
ex_view = df_AB_ex['action'].value_counts()['view']
ex_click = df_AB_ex['action'].value_counts()['click']
con_view = df_AB_con['action'].value_counts()['view']
con_click = df_AB_con['action'].value_counts()['click']

In [60]:
#Contingency Table?
contingency_table = np.array([
    (con_view, con_click),
    (ex_view, ex_click) 
    
])

contingency_table

array([[3332,  932],
       [2996,  928]])

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [51]:
#adding a column with 1s to help track our counts
df['count'] = 1
#Convert clicks into a binary variable on a user-by-user-basis
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))


Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


In [92]:
pwelches = p_value_welch_ttest(control.click, experiment.click)
pwelches

0.004466402814337078

### Yikes, that is low! Much lower than out .05! Perhaps we *CAN* reject the Null!

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [85]:
#Your code here
control_mean = control.click.mean()
expected_null_ex_cl = control_mean * len(experiment)
print(expected_null_ex_cl)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [90]:
#Your code here
n = len(experiment)
p = control_mean
var = n * p * (1-p)
std = np.sqrt(var)
print("Standard Dev:", std)

actual_ex_clicks = experiment.click.sum()
z_score = (actual_ex_clicks - expected_null_ex_cl)/std
print("Z-Score:", z_score)

Standard Dev: 24.568547907005815
Z-Score: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [91]:
#Your code here
pval = stats.norm.sf(z_score)
print(pval)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **In the sense that it shows that that the results are significant and we can reject the null, yes. They differ by quite a bit, which I find interesting. **

In [97]:
print("Prev. stat. test:", pwelches)
print("This test:", pval)

Prev. stat. test: 0.004466402814337078
This test: 0.00012486528006951198


## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.